In [1]:
import os
import shutil
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import filedialog, messagebox

# Constants
IMAGE_SIZE = (224, 224)
POTATO_CLASSES = ['Potato___Early_blight', 'Potato___Healthy', 'Potato___Late_blight']
TRAIN_DIR = r"C:\Users\pooja\Downloads\train"
VAL_DIR = r"C:\Users\pooja\Downloads\val"
MODEL_PATH = "potato_disease_model.h5"

class PotatoDiseaseClassifier:
    def __init__(self):
        self.model_path = MODEL_PATH
        self.model = None
        self.load_model()

    def load_model(self):
        try:
            print(f"Loading model from {self.model_path}")
            self.model = load_model(self.model_path)
        except Exception as e:
            print(f"Failed to load model: {e}")
            self.create_and_train_model()

    def create_and_train_model(self):
        # Data generators
        train_datagen = ImageDataGenerator(
            rescale=1./255, rotation_range=20, width_shift_range=0.2,
            height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
            horizontal_flip=True, fill_mode='nearest'
        )
        val_datagen = ImageDataGenerator(rescale=1./255)

        train_generator = train_datagen.flow_from_directory(
            TRAIN_DIR, target_size=IMAGE_SIZE, batch_size=32, class_mode='categorical')
        val_generator = val_datagen.flow_from_directory(
            VAL_DIR, target_size=IMAGE_SIZE, batch_size=32, class_mode='categorical')

        # Model
        base_model = tf.keras.applications.MobileNetV2(
            input_shape=(224, 224, 3), include_top=False, weights='imagenet')
        base_model.trainable = False

        model = tf.keras.Sequential([
            base_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(3, activation='softmax')
        ])

        print("Training model...")
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        model.fit(train_generator, validation_data=val_generator, epochs=5)

        model.save(self.model_path)
        print("Model trained and saved.")
        self.model = model

    def predict(self, image_path):
        img = Image.open(image_path).resize(IMAGE_SIZE)
        img_array = np.expand_dims(np.array(img) / 255.0, axis=0)
        predictions = self.model.predict(img_array)
        predicted_class = POTATO_CLASSES[np.argmax(predictions)]
        return predicted_class

class PotatoDiseaseApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Potato Leaf Disease Classifier")
        self.classifier = PotatoDiseaseClassifier()

        self.setup_ui()

    def setup_ui(self):
        main_frame = tk.Frame(self.root, bg="#f0f0f0", padx=20, pady=20)
        main_frame.pack()

        header_label = tk.Label(main_frame, text="Potato Leaf Disease Classifier",
                                font=("Arial", 18, "bold"), bg="#f0f0f0")
        header_label.pack(pady=10)

        instruction_label = tk.Label(main_frame, text="Upload a potato leaf image for disease classification",
                                     font=("Arial", 12), bg="#f0f0f0")
        instruction_label.pack(pady=5)

        upload_button = tk.Button(main_frame, text="Upload Image", command=self.predict_image,
                                  font=("Arial", 12), bg="#4CAF50", fg="white", padx=10, pady=5)
        upload_button.pack(pady=15)

        self.image_frame = tk.Frame(main_frame, bg="#f0f0f0")
        self.image_frame.pack(pady=10)

        self.image_label = tk.Label(self.image_frame, bg="#e0e0e0", width=300, height=300)
        self.image_label.pack()

        self.result_label = tk.Label(main_frame, text="", font=("Arial", 14), bg="#f0f0f0")
        self.result_label.pack(pady=10)

    def predict_image(self):
        file_path = filedialog.askopenfilename()
        if not file_path:
            return

        img = Image.open(file_path).resize((300, 300))
        tk_img = ImageTk.PhotoImage(img)

        self.image_label.config(image=tk_img)
        self.image_label.image = tk_img

        prediction = self.classifier.predict(file_path)
        self.result_label.config(text=f"Predicted: {prediction}")

def main():
    root = tk.Tk()
    app = PotatoDiseaseApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()

Loading model from potato_disease_model.h5


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step


In [15]:
import os

print("Subfolders inside train:")
print(os.listdir(TRAIN_DIR))

Subfolders inside train:
['Early_blight', 'Healthy', 'Late_blight']


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import class_weight
from PIL import Image, ImageTk
import tkinter as tk
from tkinter import filedialog

# Constants
IMAGE_SIZE = (224, 224)
TRAIN_DIR = r"C:\Users\pooja\Downloads\train"
VAL_DIR = r"C:\Users\pooja\Downloads\val"
MODEL_PATH = "potato_disease_model_trained.h5"
POTATO_CLASSES = sorted(os.listdir(TRAIN_DIR))  # ['Early_blight', 'Healthy', 'Late_blight']

class PotatoDiseaseClassifier:
    def __init__(self):
        self.model_path = MODEL_PATH
        self.model = None
        self.train_and_save_model()

    def train_and_save_model(self):
        print(" Training new model...")
        # Data Generators
        train_gen = ImageDataGenerator(
            rescale=1./255, rotation_range=20, width_shift_range=0.2,
            height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
            horizontal_flip=True, fill_mode='nearest'
        )
        val_gen = ImageDataGenerator(rescale=1./255)

        train_data = train_gen.flow_from_directory(
            TRAIN_DIR, target_size=IMAGE_SIZE, batch_size=32, class_mode='categorical')
        val_data = val_gen.flow_from_directory(
            VAL_DIR, target_size=IMAGE_SIZE, batch_size=32, class_mode='categorical')

        print("Class indices:", train_data.class_indices)

        # Compute class weights
        all_labels = []
        for i, cls in enumerate(POTATO_CLASSES):
            all_labels += [i] * len(os.listdir(os.path.join(TRAIN_DIR, cls)))

        weights = class_weight.compute_class_weight(class_weight='balanced',
                                                    classes=np.unique(all_labels),
                                                    y=all_labels)
        class_weights = dict(enumerate(weights))

        # Model
        base_model = tf.keras.applications.MobileNetV2(
            input_shape=(224, 224, 3), include_top=False, weights='imagenet')
        base_model.trainable = False

        model = tf.keras.Sequential([
            base_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(len(POTATO_CLASSES), activation='softmax')
        ])

        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        model.fit(train_data, validation_data=val_data, epochs=5, class_weight=class_weights)

        model.save(self.model_path)
        self.model = model
        print(" Model trained and saved.")

    def predict(self, image_path):
        img = Image.open(image_path).resize(IMAGE_SIZE)
        img_array = np.expand_dims(np.array(img) / 255.0, axis=0)
        predictions = self.model.predict(img_array)[0]

        for i, prob in enumerate(predictions):
            print(f"{POTATO_CLASSES[i]}: {prob:.4f}")

        predicted_index = np.argmax(predictions)
        return POTATO_CLASSES[predicted_index]


class PotatoDiseaseApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Potato Leaf Disease Classifier")
        self.classifier = PotatoDiseaseClassifier()

        self.setup_ui()

    def setup_ui(self):
        main_frame = tk.Frame(self.root, bg="#f0f0f0", padx=20, pady=20)
        main_frame.pack()

        tk.Label(main_frame, text="Potato Leaf Disease Classifier", font=("Arial", 18, "bold"), bg="#f0f0f0").pack(pady=10)
        tk.Label(main_frame, text="Upload a potato leaf image for disease classification", font=("Arial", 12), bg="#f0f0f0").pack(pady=5)

        tk.Button(main_frame, text="Upload Image", command=self.predict_image,
                  font=("Arial", 12), bg="#4CAF50", fg="white", padx=10, pady=5).pack(pady=15)

        self.image_frame = tk.Frame(main_frame, bg="#f0f0f0")
        self.image_frame.pack(pady=10)

        self.image_label = tk.Label(self.image_frame, bg="#e0e0e0", width=300, height=300)
        self.image_label.pack()

        self.result_label = tk.Label(main_frame, text="", font=("Arial", 14), bg="#f0f0f0")
        self.result_label.pack(pady=10)

    def predict_image(self):
        file_path = filedialog.askopenfilename()
        if not file_path:
            return

        img = Image.open(file_path).resize((300, 300))
        tk_img = ImageTk.PhotoImage(img)
        self.image_label.config(image=tk_img)
        self.image_label.image = tk_img

        prediction = self.classifier.predict(file_path)
        self.result_label.config(text=f"Predicted: {prediction}")

def main():
    root = tk.Tk()
    app = PotatoDiseaseApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()

 Training new model...
Found 2400 images belonging to 3 classes.
Found 600 images belonging to 3 classes.
Class indices: {'Early_blight': 0, 'Healthy': 1, 'Late_blight': 2}


C:\Users\pooja\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
75/75 ━━━━━━━━━━━━━━━━━━━━ 56s 681ms/step - accuracy: 0.7965 - loss: 0.5285 - val_accuracy: 0.9400 - val_loss: 0.1606
Epoch 2/5
75/75 ━━━━━━━━━━━━━━━━━━━━ 43s 567ms/step - accuracy: 0.9580 - loss: 0.1291 - val_accuracy: 0.9817 - val_loss: 0.0742
Epoch 3/5
75/75 ━━━━━━━━━━━━━━━━━━━━ 45s 605ms/step - accuracy: 0.9585 - loss: 0.1089 - val_accuracy: 0.9617 - val_loss: 0.1112
Epoch 4/5
75/75 ━━━━━━━━━━━━━━━━━━━━ 46s 611ms/step - accuracy: 0.9722 - loss: 0.0693 - val_accuracy: 0.9767 - val_loss: 0.0738
Epoch 5/5
75/75 ━━━━━━━━━━━━━━━━━━━━ 45s 601ms/step - accuracy: 0.9721 - loss: 0.0787 - val_accuracy: 0.9867 - val_loss: 0.0492


 Model trained and saved.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 802ms/step
Early_blight: 0.9979
Healthy: 0.0000
Late_blight: 0.0021
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Early_blight: 0.0000
Healthy: 0.9987
Late_blight: 0.0013
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Early_blight: 0.9964
Healthy: 0.0000
Late_blight: 0.0036
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Early_blight: 0.0000
Healthy: 0.0003
Late_blight: 0.9997
